## introduction
### problem statement
Every day always more people experience the opportunity of training overywhere: outdoor on a park, in a garage, in our bedroom etc. Anyway this possibility of freedom led to a great issue commont to every uncommon athlets: the lack of feedbacks.
infact, while in a gym is easy receive a examination on own exercise esecution, by a trainer or by another athlet, on own bedroom we don't have this possibility. lack of feedback on exercise esecution led to learn improper esecutin, bring to weak performances, injuries etc

### state of art 
to evaluete own proper execution, athlets sometimes use to record their performarce in order to self esaminate them and find what is wrong. Anyway this process is tedios becouse it involve the need to find a good place to put the smartphone,record the clip, skip all the preparation of exercise, find the most significative point on the esecution, examinate them and finaly understaining whats wrong for correcting mistakes on the next repetition.
Do this proces for every rep, for every exercise, in time with rest time (usualy in the order of 1 minute), can be annoing and most athlets preferes to not be distracted and keep attention on their work out.

### proposed solution 
my idea is to make all this process agile for the atlets, developing a computer vision pipeline that is able  to take the recordered clip, extrapolate the most important keyframe for the athlets, discard all the unnecessary moment to be keep user attention on the most important moment and give to final user a edited clipp to fast evaluate own performances


# Step O colab initialization

In [1]:
!git clone https://github.com/AlfaranoAndrea/workuout_keyframe.git
%cd workuout_keyframe 
!gdown https://drive.google.com/drive/folders/148Mhwpt9E89_gLMAJC4S45DiVv83Xz54?usp=share_link --folder -O input_video


Cloning into 'workuout_keyframe'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 134 (delta 51), reused 98 (delta 38), pack-reused 4
Receiving objects: 100% (134/134), 5.68 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/workuout_keyframe
Retrieving folder list
Processing file 1KC4dLg3R-RVVxKlEeXwidfX6CMcALzEX military.mp4
Processing file 1VMJWvQ_dWtwkw6uaQUAV8s8zrFE1n6u6 poor_execution.mp4
Processing file 1DILeSomSV_LVZK6O_LSDVQ7yhz52KDE_ pullup.mp4
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1KC4dLg3R-RVVxKlEeXwidfX6CMcALzEX
To: /content/workuout_keyframe/input_video/military.mp4
100% 58.5M/58.5M [00:03<00:00, 16.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VMJWvQ_dWtwkw6uaQUAV8s8zrFE1n6u6
To: /content/workuout_keyframe/input_video/poo

In [2]:
!mkdir -p /models/checkpoint/
!wget "drive.google.com/u/3/uc?id=1iAKv2S1OE3YpwgwX43bKZHxBAlluL9Y9&export=download&confirm=yes" -O /models/checkpoint/flownets.pth.tar.pth

--2023-02-16 16:22:51--  http://drive.google.com/u/3/uc?id=1iAKv2S1OE3YpwgwX43bKZHxBAlluL9Y9&export=download&confirm=yes
Resolving drive.google.com (drive.google.com)... 142.251.2.100, 142.251.2.101, 142.251.2.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.100|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/u/3/uc?id=1iAKv2S1OE3YpwgwX43bKZHxBAlluL9Y9&export=download&confirm=yes [following]
--2023-02-16 16:22:51--  https://drive.google.com/u/3/uc?id=1iAKv2S1OE3YpwgwX43bKZHxBAlluL9Y9&export=download&confirm=yes
Connecting to drive.google.com (drive.google.com)|142.251.2.100|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1iAKv2S1OE3YpwgwX43bKZHxBAlluL9Y9&export=download&confirm=yes [following]
--2023-02-16 16:22:51--  https://drive.google.com/uc?id=1iAKv2S1OE3YpwgwX43bKZHxBAlluL9Y9&export=download&confirm=yes
Reusing existing connection to

In [ ]:
!python3 downloadSintel.py

############### DOWNLOADING SINTEL DATA ###############
5.50MKB [04:48, 19.0kKB/s]               
############### UNZIPPING SINTEL DATA ###############


In [4]:
!pip install pytorch_lightning --quiet
!pip install flowiz --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 16.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user


# First part: Deep Learning

In [5]:
import numpy as np
import torch
import pytorch_lightning as pl

from torchvision import datasets
import matplotlib.pyplot as plt
import flowiz as fz

from dataset import  FlowDataModule
from models.flowNetS import FlowNetS
from models.raft import RAFT

from utilities.videoUtilities import extract_frames, saveVideo
from utilities.flowUtilities import computeFlow, flowVideo
from utilities.keyFramesUtilities import find_center_of_gravity, save_key_frames
from utilities.signalUtilities import  movingAverage, findKeyPoints, normalizeZeroMean

ModuleNotFoundError: ignored

# datasets

let's visualize how torch vison dataset FlyingChairs works

In [ ]:
loadedDataset= datasets.Sintel(root='./dataset',
                              split = "train", 
                              #transforms = dataModulo.dataset_transformation,
                              pass_name = "clean"
                                )

In [ ]:
"""
dataset gives back triplets [image1,image2, targetFlow]
"""

triplet=loadedDataset[10]
img1,img2,flow= triplet

img1=np.asarray(img1)
img2=np.asarray(img2)

print (f"image1 in dataset has shape: {img1.shape}")
print (f"image2 in dataset has shape: {img2.shape}")

print (f"flow image in dataset has shape: {flow.shape}")
flow=np.transpose (flow,(1,2,0))
print (f"flow image in dataset has this new shape: {flow.shape}")

flow=fz.convert_from_flow(flow) 

plt.figure(figsize=(15,10))
plt.subplot(1, 3, 1)
plt.imshow(img1)
plt.subplot(1, 3, 2)
plt.imshow(img2)
plt.subplot(1, 3, 3)
plt.imshow(flow)



## model training

In [ ]:
save = torch.load("./models/checkpoint/flownets.pth.tar.pth")#["state_dict"]
model  = FlowNetS(save)
data=FlowDataModule(batch_size =1, workers=32,selected_dataset ="Sintel")

trainer = pl.Trainer(accelerator= "gpu" if torch.cuda.is_available() else "cpu", 
                     max_epochs=20,             
                     auto_lr_find=False, 
                     auto_scale_batch_size=False)

In [ ]:
trainer.fit(model,data )
trainer.test(model, data)

# Second part: Computing Pipeline

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
save = torch.load("./models/checkpoint/flownets.pth.tar.pth")#["state_dict"]
model  = FlowNetS(save)
model.to(device)
video_path= "./input_video/pullup.mp4"
temp_extracted_frames="./temp/Frames/"
temp_processed_flow_frames="./temp/processedFloFrames/"

In [ ]:
fps=30
step=6
#fps=int(extract_frames(video_path))
flowVideo(
    path=temp_extracted_frames,
    model=model,
    device=device,
    step=step)
saveVideo(temp_processed_flow_frames,"pippo", fps=fps)

In [ ]:
# 3B- Find miving cordinates of center of gravity of our athlet
x,y=find_center_of_gravity(temp_processed_flow_frames)  
#saveVideo(comp,"PullUp_mass",bw=True)

## SIGNAL ANALISYS

In [ ]:
fig, ax = plt.subplots()
ax.plot(y)
plt.show()

In [ ]:
# 2- mobile average
#K=[0.25,0.25,0.25,0.25]


#avg=np.convolve(K,y)
avg= movingAverage(y,k=int(fps/step))
normalized= normalizeZeroMean(avg)
fig, ax = plt.subplots()
ax.plot(avg)
plt.show()

In [ ]:
# 3- find peacks positive
peaks= findKeyPoints(normalized, distance=int(fps/step))


np.diff(peaks)
plt.figure(figsize=(15,10))
plt.plot(normalized)
plt.plot(peaks, normalized[peaks], "x")
plt.show()



In [ ]:
save_key_frames(
    keyFramesList=peaks, 
    frames_path=temp_extracted_frames,
    threshold=12, 
    near=5, 
    slowmotion=2, 
    speedUp=4, 
    title="final.avi")